In [14]:
import os
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from facenet_pytorch import MTCNN
from torchvision.models import resnet50, ResNet50_Weights
import torchvision.transforms as T
from PIL import Image

In [15]:
preprocess = T.Compose([
    T.ToTensor(),
    T.Resize(224),
    T.Normalize(
       mean=[0.485, 0.456, 0.406],
       std=[0.229, 0.224, 0.225]
   )
])

In [16]:
import torch.nn as nn
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        base_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.model = nn.Sequential(*list(base_model.children())[:-1])

    def forward(self, x):
        return self.model(x)

In [18]:
feature_extractor = Model()
feature_extractor.eval()

Model(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256,

In [19]:
# Code for training the model on image dataset

features_lst=[]
filenames = pickle.load(open('filenames.pkl','rb'))
for filename in filenames:
    image = Image.open(filename).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        features = feature_extractor(input_tensor)
    features_lst.append(features.flatten())

In [20]:
# store the feature list in a file
pickle.dump(features_lst,open("embedding.pkl","wb"))